In [30]:
# Import packages
import poreana as pa
import porems as pms
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as widgets


import MDAnalysis as mda
#import nglview as nv

from IPython.display import display, Markdown, Latex


In [73]:
# Set strings to the files in data set
def set_strings(molecule):
    strings = {}
    # Strcuture for molecule
    strings["gro_mol"] = "Molecules/{}/{}.gro".format(molecule,molecule)

    # Strings for pure rectangular systems
    strings["dens_box"] = "Pure/{}/box_rectangular_{}_density.h5".format(molecule,molecule)
    strings["mc_box"] = "Pure/{}/box_rectangular_{}_diffusion_smoluchowski.h5".format(molecule,molecule)
    # Strings for pore system
    strings["dens"] = "Pore/{}/pore_ideal_{}_density.h5".format(molecule,molecule)
    strings["bin"] = "Pore/{}/pore_ideal_{}_diffusion_einstein.h5".format(molecule,molecule)
    strings["mc"] = "Pore/{}/pore_ideal_{}_diffusion_smoluchowski.h5".format(molecule,molecule)

    #Strcuture for pore system
    strings["gro_pore"] = "Pure/{}/pure_{}_01k.gro".format(molecule,molecule)

    # Strings for amoprh pore systems (if exisit)
    if molecule in ["cyclopentane","hexane","heptan"]:
        strings["dens"] = "Pore/{}/pore_amorph_{}_density.h5".format(molecule,molecule)
        strings["bin"] = "Pore/{}/pore_amorph_{}_diffusion_einstein.h5".format(molecule,molecule)
        strings["mc"] = "Pore/{}/pore_amorph_{}_diffusion_smoluchowski.h5".format(molecule,molecule)
    return strings

In [77]:
# Display molcule and system
def display_molecule(strings):
    display(Markdown('# Molecule'))
    mol = mda.Universe(strings["gro_mol"])
    view = nv.show_mdanalysis(mol)
    display(view)
    display(Markdown('# System'))
    mol = mda.Universe(strings["gro_pore"])
    view = nv.show_mdanalysis(mol)
    display(view)

In [14]:
# Plot density function
def plot_density(strings):
    display(Markdown('# Density'))
    plt.figure(figsize=(14,4))
    plt.subplot(1,2,1)
    plt.title("Reservoir density")
    dens = pa.density.bins(strings["dens"], is_print=False)
    plt.plot(dens["sample"]["data"]["ex_width"],[den for den in dens["num_dens"]["ex"]], label = "Reservoir Density")
    plt.plot(dens["sample"]["data"]["ex_width"],[dens["mean"]["ex"] for i in dens["num_dens"]["ex"]], linestyle = "--", label = "Mean Reservoir Density")
    plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
    plt.xlabel("Reservoir length (nm)")
    plt.xlim([0,10])
    plt.subplot(1,2,2)
    plt.title("Radial Pore Density")
    dens = pa.density.bins(strings["dens"], is_print=False)
    plt.plot(dens["sample"]["data"]["in_width"][:-1],[den for den in dens["num_dens"]["in"]], label = "Pore Density")
    plt.plot(dens["sample"]["data"]["ex_width"][:-1],[dens["mean"]["ex"] for i in dens["num_dens"]["ex"][:-1]], linestyle = "--", label = "Mean Reservoir Density")
    plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label= "Sinanol O")
    plt.ylabel("Density (molecules $\mathrm{nm}^{-1}$)")
    plt.xlabel("Distance from pore center (nm)")
    #plt.title(molecule)
    plt.legend()
    plt.xlim([0,2.5])
    plt.tight_layout()
    plt.show()

In [28]:
# Diffusion (MC)
def plot_mc_diffusion(strings):
    plt.figure(figsize=(14,4))
    # Plot bin diffusion
    plt.subplot(1,2,1)
    plt.title("Bin diffusion")
    mc_box = pa.diffusion.mc_fit(strings["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
    diff_bin = pa.diffusion.bins(strings["bin"])
    ax  =sns.lineplot(x=np.linspace(0,2.51,100),y=[mc_box[0] for i in range(100)], linestyle="--")
    pa.diffusion.bins_plot(diff_bin)
    plt.axvspan(xmin=2.1, xmax=2.5, facecolor="grey", alpha=0.3, label = "Silanol O")
    plt.xlim([0,2.5])
    ax.set_ylim(bottom=0)
    plt.text(0.05, 0.1, 'NVT Diffusion : D =' + "%.2f" % (0) +"$\\cdot 10^{-9}$ \nReservoir Diffusion : D =" + "%.2f" % (0) + "$\\cdot 10^{-9}$ \nPore Diffusion : D =" + "%.2f" % (0) +"$\\cdot 10^{-9}$"  ,  bbox={'facecolor': 'grey', 'alpha': 0.2, 'pad': 4})
    

    plt.subplot(1,2,2)
    display(Markdown('# Diffusion Profile'))
    plt.title("MC Diffusion")
    mc_box = pa.diffusion.mc_fit(strings["mc_box"], len_step = [10,20,30,40,50,60], is_plot=False, is_print=False)
    mc_pore = pa.diffusion.mc_profile(strings["mc"], len_step = [10,20,30,40,50,60], kwargs={"label": "Pore System"})
    pore = pa.diffusion.mc_fit(strings["mc"], len_step = [10,20,30,40,50,60], section = [15,19.7], is_plot=False, is_print=False)
    res = pa.diffusion.mc_fit(strings["mc"], len_step = [10,20,30,40,50,60], section = [0,7], is_plot=False, is_print=False)
    sns.lineplot(x=np.linspace(0,30.1,100),y=[mc_box[0] for i in range(100)], linestyle="--", label="Box Diffusion")
    plt.text(21, min(mc_pore[0]), 'NVT : D = ' + "%.2f" % (mc_box[1]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nRes  : D = " + "%.2f" % (res[0]) + "$\\cdot 10^{-9}\ \\frac{m^2}{s}$ \nPore : D = " + "%.2f" % (pore[0]) +"$\\cdot 10^{-9}\ \\frac{m^2}{s}$"  ,  bbox={'facecolor': 'grey', 'alpha': 0.8, 'pad': 4})

    # Plot area
    plt.axvspan(xmin=0, xmax=10, facecolor="grey", alpha=0.3, label = "Reservoir")
    plt.axvspan(xmin=20, xmax=31, facecolor="grey", alpha=0.3)
    plt.axvspan(xmin=15, xmax=19.7, facecolor="red", alpha=0.3, label = "Evaluated")
    plt.axvspan(xmin=23, xmax=31, facecolor="red", alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [56]:
# Graphic user interface
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import nglview as nv



#input_widget = widgets.HBox([drop_group,drop])
def view_results(molecule):
    import poreana as pa
    import matplotlib.pyplot as plt
    import MDAnalysis as mda
    from IPython.display import display, Markdown, Latex


    strings = set_strings(molecule)
    display_molecule(strings)
    plot_density(strings)
    plot_mc_diffusion(strings)


def dashboard():
    
    dict = {}
    dict["alcohol"] =   ["methanol","ethanol","1-propanol","1-butanol"]
    dict["aromatic"] =   ["benzene","toluene","pyrrol","pyridine"]
    dict["alkane"] =   ["hexane","heptane","cyclopentane","cylcohexane",]
    dict["others"] =   ["water","tetrahydrofuran"]

    display(Markdown("# Overview   \n  The data set contains the following systems at a Temperature $T=295 K$: \n"))
    display(Markdown("|Alcohol   |Aromatic|Alkene      |\n"    "|:---------|:-------|:-----------|\n"    "|Methanol  |Benzene |Cyclopentane|\n"    "|Ethanol   |Toluene |Cyclohexane |\n"    "|1-Propanol|Pyrole  |Hexane      |\n"    "|1-Butanol |Pyridine|Heptane     |\n"    "\n"    "\n"))

    display(Markdown("# Set molecule"))
    # Define dropdowns
    drop_group = widgets.Dropdown(
    options=dict.keys(),
    )



    drop = widgets.Dropdown(options=["methanol","ethanol","1-propanol","1-butanol","benzene","toluene","pyrrole","pyridine","hexane","heptane","cyclopentane","cylcohexane", "water","tetrahydrofuran"]
    )



    a = widgets.HBox([drop_group,drop,])

    interact(drop = drop,drop_group = drop_group)
    interact(view_results,molecule = drop)



In [76]:
# Display dashboard
dashboard()

# Overview   
  The data set contains the following systems at a Temperature $T=295 K$: 


|Alcohol   |Aromatic|Alkene      |
|:---------|:-------|:-----------|
|Methanol  |Benzene |Cyclopentane|
|Ethanol   |Toluene |Cyclohexane |
|1-Propanol|Pyrole  |Hexane      |
|1-Butanol |Pyridine|Heptane     |




# Set molecule

interactive(children=(Dropdown(description='molecule', options=('methanol', 'ethanol', '1-propanol', '1-butano…